## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.drop(["EIN","NAME"], axis = 1, inplace=True)

In [3]:
# Determine the number of unique values in each column. Also satisfying OCD with string formatting
for column in application_df:
    series = len(application_df[column].unique())
    if column == "SPECIAL_CONSIDERATIONS":
        print(f"{column:<10}:{series:>4}")
    else:
        print(f"{column:<16}:{series:>10}")

APPLICATION_TYPE:        17
AFFILIATION     :         6
CLASSIFICATION  :        71
USE_CASE        :         5
ORGANIZATION    :         4
STATUS          :         2
INCOME_AMT      :         9
SPECIAL_CONSIDERATIONS:   2
ASK_AMT         :      8747
IS_SUCCESSFUL   :         2


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Attempt 1 at Optimization, increasing bins available
application_types_to_replace = ["T25", "T14", "T29", "T15", "T17"]

In [6]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_bins = application_df["CLASSIFICATION"].value_counts()
classification_bins.loc[classification_bins > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Attempt at Optimization, increasing bins available
classification_index =  classification_bins.index
keep_values = ["C1000", "C2000","C1200", "C3000", "C2100", "C7000", "C1700", "C4000", "C5000", "C1270", "C2700"]

In [9]:
classifications_to_replace = []

for item in classification_index:
    if item not in keep_values:
        classifications_to_replace.append(item)
    else:
        pass

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [19]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_variable_df = pd.get_dummies(data = application_df, columns = ["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS", "ASK_AMT"])
dummy_variable_df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,...,ASK_AMT_1665460552,ASK_AMT_1736232349,ASK_AMT_1893400128,ASK_AMT_2264109450,ASK_AMT_2310256039,ASK_AMT_3391919220,ASK_AMT_4653011914,ASK_AMT_5591584994,ASK_AMT_8556638692,ASK_AMT_8597806340
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Split our preprocessed data into our features and target arrays
X = dummy_variable_df.drop("IS_SUCCESSFUL", axis = 1)
y = dummy_variable_df["IS_SUCCESSFUL"]
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 25)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Attempting to Optimize Model

In [22]:
# Attempt 1, adding more layers
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 500
hidden_nodes_layer2 = 500
hidden_nodes_layer3 = 450
hidden_nodes_layer4 = 450
hidden_nodes_layer5 = 400
nn = tf.keras.models.Sequential()


nn.add(tf.keras.layers.Dense(input_dim = number_input_features, units=hidden_nodes_layer1, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 500)               4400000   
                                                                 
 dense_21 (Dense)            (None, 500)               250500    
                                                                 
 dense_22 (Dense)            (None, 450)               225450    
                                                                 
 dense_23 (Dense)            (None, 450)               202950    
                                                                 
 dense_24 (Dense)            (None, 400)               180400    
                                                                 
 dense_25 (Dense)            (None, 1)                 401       
                                                                 
Total params: 5,259,701
Trainable params: 5,259,701
No

In [24]:
# Testing the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled,y_train,epochs=5)

Epoch 1/5
804/804 [==============================] - 48s 59ms/step - loss: 0.3879 - accuracy: 0.8157
Epoch 2/5
804/804 [==============================] - 45s 56ms/step - loss: 0.4047 - accuracy: 0.8145
Epoch 3/5
804/804 [==============================] - 47s 58ms/step - loss: 0.3900 - accuracy: 0.8150
Epoch 4/5
804/804 [==============================] - 45599s 57s/step - loss: 0.3957 - accuracy: 0.8145
Epoch 5/5
804/804 [==============================] - 45s 57ms/step - loss: 0.3895 - accuracy: 0.8157


In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6252 - accuracy: 0.7138 - 1s/epoch - 5ms/step
Loss: 0.6251592040061951, Accuracy: 0.7138192653656006


In [26]:
# Attempt 2, removing more hyper parameters
X = dummy_variable_df.drop(["IS_SUCCESSFUL", "SPECIAL_CONSIDERATIONS_N", "SPECIAL_CONSIDERATIONS_Y", "STATUS"], axis = 1)
y = dummy_variable_df["IS_SUCCESSFUL"]
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 25)


In [27]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [35]:
# Attempt 2, creating less layers with denser connections
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 3125
hidden_nodes_layer2 = 625
hidden_nodes_layer3 = 125

nn = tf.keras.models.Sequential()


nn.add(tf.keras.layers.Dense(input_dim = number_input_features, units=hidden_nodes_layer1, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))


nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 3125)              27490625  
                                                                 
 dense_17 (Dense)            (None, 625)               1953750   
                                                                 
 dense_18 (Dense)            (None, 125)               78250     
                                                                 
 dense_19 (Dense)            (None, 1)                 126       
                                                                 
Total params: 29,522,751
Trainable params: 29,522,751
Non-trainable params: 0
_________________________________________________________________


In [36]:
# Attempt 2 Model Test
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled,y_train,epochs=5)

Epoch 1/5
804/804 [==============================] - 246s 303ms/step - loss: 0.6094 - accuracy: 0.6947
Epoch 2/5
804/804 [==============================] - 246s 305ms/step - loss: 0.4240 - accuracy: 0.8038
Epoch 3/5
804/804 [==============================] - 239s 298ms/step - loss: 0.4039 - accuracy: 0.8120
Epoch 4/5
804/804 [==============================] - 242s 301ms/step - loss: 0.3977 - accuracy: 0.8122
Epoch 5/5
804/804 [==============================] - 243s 303ms/step - loss: 0.3986 - accuracy: 0.8126


In [37]:
# Attempt 2 Evaluation
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 5s - loss: 0.6141 - accuracy: 0.7187 - 5s/epoch - 20ms/step
Loss: 0.6141059398651123, Accuracy: 0.7187172174453735


In [58]:
# Attempt 3 Keeping all parameters the same but using different activation functions
X = dummy_variable_df.drop(["IS_SUCCESSFUL"], axis = 1)
y = dummy_variable_df["IS_SUCCESSFUL"]
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 25) 

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [119]:
# Attempt 3, using constant amount of nodes with more layers and declining nodes
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer = 1500
hidden_nodes_layer2 = 1500
hidden_nodes_layer3 = 1200
hidden_nodes_layer4 = 1200
hidden_nodes_layer5 = 600
hidden_nodes_layer6 = 600



nn = tf.keras.models.Sequential()


nn.add(tf.keras.layers.Dense(input_dim = number_input_features, units=hidden_nodes_layer1, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="elu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="elu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="elu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation="relu"))



nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn.summary()

Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_231 (Dense)           (None, 500)               4400000   
                                                                 
 dense_232 (Dense)           (None, 1500)              751500    
                                                                 
 dense_233 (Dense)           (None, 1200)              1801200   
                                                                 
 dense_234 (Dense)           (None, 1200)              1441200   
                                                                 
 dense_235 (Dense)           (None, 600)               720600    
                                                                 
 dense_236 (Dense)           (None, 600)               360600    
                                                                 
 dense_237 (Dense)           (None, 1)               

In [120]:
# Attempt 3 Model Test
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled,y_train,epochs=5)

Epoch 1/5
804/804 [==============================] - 89s 105ms/step - loss: 0.6103 - accuracy: 0.6813
Epoch 2/5
804/804 [==============================] - 84s 105ms/step - loss: 0.5352 - accuracy: 0.7510
Epoch 3/5
804/804 [==============================] - 80s 100ms/step - loss: 0.4130 - accuracy: 0.8090
Epoch 4/5
804/804 [==============================] - 80s 100ms/step - loss: 0.4022 - accuracy: 0.8106
Epoch 5/5
804/804 [==============================] - 85s 106ms/step - loss: 0.3998 - accuracy: 0.8123


In [121]:
# Attempt 3 Evaluation
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 3s - loss: 0.6353 - accuracy: 0.7062 - 3s/epoch - 11ms/step
Loss: 0.6353119015693665, Accuracy: 0.7062390446662903


In [122]:
# Export our model to HDF5 file
nn.save_weights("AlphabetSoupCharity_Optimization.hdf5")